<center><h3>Machine Learning (aprendizado de máquina) de doadores de sangue em 2007</h3><br>
Objetivo: prever de acordo com os dados anteriores doaram no mês de março em 2007, ver descrição completa no relatório.</center>

 Fonte de dados: https://archive.ics.uci.edu/ml/datasets/Blood+Transfusion+Service+Center

In [1]:
#Importando as bibliotecas necessárias, pandas e numpy.
import pandas as pd
import numpy as np

In [2]:
#chamando nossa base pelo pandas, essa base foi baixada no meus documentos de computador:
arq = pd.read_csv('transfusion.data')
arq.index.rename('id_registro', inplace=True)
arq.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
id_registro,,,,,
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [3]:
##Verificação se nossos dados tem dados negativos entre as linhas e colunas:
arq[arq.iloc[:, 0:]<0].sum()


Recency (months)                              0.0
Frequency (times)                             0.0
Monetary (c.c. blood)                         0.0
Time (months)                                 0.0
whether he/she donated blood in March 2007    0.0
dtype: float64

In [4]:
#Agora veirificando se possui dados nulos, NaN:
arq.isnull().sum()

Recency (months)                              0
Frequency (times)                             0
Monetary (c.c. blood)                         0
Time (months)                                 0
whether he/she donated blood in March 2007    0
dtype: int64

In [5]:
##previsores, estes nos indicam quais iremos usar para prever e classe, este são os que vamos prever
previsores = arq.iloc[:, 0:4].values
classe = arq.iloc[:,4].values

In [6]:
##Localização de registros com dados ausentes e preenchimento por default
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'most_frequent')
imputer = imputer.fit(previsores)
previsores = imputer.transform(previsores)

In [7]:
##Escalonamento de atributo para tornar a escala mais próxima entre os dados
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

In [8]:
##Divisão para teste e treinamento da classe e previsores, importante dizer que utilizei um teste em 25% e um random para aleatoriedade 0
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size = 0.25, random_state=0)

In [9]:
##utilizando o metodo do teorema bayes(probabilístico)
from sklearn.naive_bayes import GaussianNB
classificador = GaussianNB()
classificador.fit(previsores_treinamento, classe_treinamento)

#Vou utilizar de forma difinitiva os previsores_teste 
previsoes = classificador.predict(previsores_teste)


#mas apliquei sobre nosso conjunto de dados apenas para visualização posterior aqui e nada mais.
previs = classificador.predict(previsores)

#uma nova coluna foi erada com nosso resultado:
arq['previstos'] = previs
arq.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007,previstos
id_registro,,,,,,
0,2,50,12500,98,1,1
1,0,13,3250,28,1,1
2,1,16,4000,35,1,1
3,2,20,5000,45,1,1
4,1,24,6000,77,0,1


In [10]:
##fazendo a matriz de acertos e erros:
from sklearn.metrics import confusion_matrix, accuracy_score
precisao = accuracy_score(classe_teste,previsoes)
matriz = confusion_matrix(classe_teste,previsoes)

#Zero significa não doou e 1 significa doou, se lê "era 0 e aplicou 0: 124"; "era 0  aplicou 1: 8"; "era 1 aplicou 1: 13"
#Grau de precisão do nosso modelo: 73,26% :
visu_matriz = pd.DataFrame(matriz)
visu_matriz

,0,1
0,124,8
1,42,13
